 **Run the next cell to set up the functions that will rename each image according to the detected barcode.** 
    * Note that you will need to have the following packages installed: [numpy](https://numpy.org/install/), [pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/install.html), [pillow](https://pillow.readthedocs.io/en/stable/installation.html), and [pyzbar](https://pypi.org/project/pyzbar/)
    * You will need to have this notebook saved in the same folder as the excel sheet matching each bar code to the corresponding plate information, which should have column names matching the example provided ('Barcode', 'Plate', 'Condition'). It will not impact the function of this script if there are additional columns.
    * All images to be processed should be saved in the same folder. The path to this folder will be one of the arguments provided to the re-naming function
    * This function is currently set up to handle jpeg images
    * The function takes an optional argument 'key', which will be included in every new image name. If no key is entered, this field is left blank.

In [ ]:
# --- import packages
import numpy as np
import pandas as pd
import glob
import os

from PIL import Image
from PIL.ExifTags import TAGS
from pyzbar.pyzbar import decode

# --- define functions

def get_field (exif,field):
    for (k,v) in exif.items():
        if TAGS.get(k) == field:
            return v

def rename_images(image_folder_path, mastersheet, key =''):
    cwd = os.getcwd() # save path to current working directory
    df = pd.read_excel (mastersheet) # read in plate info -- master sheet must be saved in current working directory
    s = df['Barcode'] # build data frame from master sheet
    
    os.chdir(image_folder_path) # move to image folder
    image_filenames = glob.glob('*.jpg') + glob.glob('*.JPG') # make a list of all .jpg files
    print('Processing ' + str(len(image_filenames)) + ' images...') # print number of images detected to the screen
    
    for filename in image_filenames:
        image = Image.open(filename)
        im = np.array(image)
        all_the_info = decode(im) # detect and decode barcode
        if not all_the_info: 
            continue # if barcode is not legible skip this image
        barcode = all_the_info[0].data
        d = barcode.decode('ASCII') # translate to digit
        try:
            image_row = df.loc[s==int(d)] # find which row this barcode corresponds to in the mastersheet
            info = image_row.iloc[0] 
            exif_data = image._getexif()
            DT = get_field(exif_data,'DateTime')
            Time = DT.split()[1]
            Time = Time.replace(':','-')
            # new image name contains the plate info, condition info, key (24H, 48H, 72H, SW, HW), and time info
            new_name = info.Plate+'_'+info.Condition+'_'+key+'_'+Time+'.jpg'
            dst = new_name
            src = filename
            # rename image in place
            os.rename(src,dst)
        except: 
            print('Barcode'+str(d)+' is not on the master sheet.') # print out missing barcodes, if any
    os.chdir(cwd)


**Run the next cell to rename the images contained in ImageFolder according to their barcode.** 

The new name will be 'Plate_Condition_key_hr-min-sec.jpg'. The only output to the screen will be the number of images in the folder that are being processed. If a barcode is detected that is not included on the master sheet this will also be reported. 

In [ ]:
image_folder_path = r'C:\Users\User\Desktop\ImageFolder' # replace with path to image folder
mastersheet = 'Plate Name Master Sheet.xlsx' # replace with name of master sheet, saved in current working directory
key = 'SW' # Images were saved in folders according to the time point at which they were taken. 
           # This info is encoded in the new image names as
           # '24H', '48H', '72H', 'SW' (for soft wash), or 'HW' (for hard wash)
        
rename_images(image_folder_path,mastersheet,key = key)